# Refine the Data

Create the items, users and interaction matrix

- **Items**: item id + metadata features
- **Users**: User id + metadata features
- **Interaction**: Explicit Matrix, Implicit Matrix

In [1]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import re
from reco.preprocess import remove_year

## Items

For the items, we will keep the following:
- From `items_raw` dataframe
  - movie_id
  - title
  - year (release)
  - genre categories
- From `item_features` dataframe
  - overview
  - language (original_language)
  - runtime
  - vote_average
  - vote_count

In [3]:
items_raw = pd.read_csv("data/items_raw.csv")
item_features = pd.read_csv("data/item_features.csv")

*1. Get `year` from `release_title`* 

In [4]:
items_raw["release_date"] = pd.to_datetime(items_raw.release_date, infer_datetime_format=True)
items_raw["year"] = items_raw.release_date.apply(lambda x: str(x.year))

*2. Drop `imdb_url`,  `video_release_date` & `release_date`*

In [5]:
items_main = items_raw.drop(['video_release_date', 'release_date', 'imdb_url'], axis=1).copy()

In [6]:
# Match Whitespace + ( + YEAR + )
# regex_year = re.compile(r'\s\(\d{4}\)')
# items["movie"] = items.title.str.replace(regex_year, "")
# items["movie"] = items.movie.str.strip()

*3. Get the additional features from the item_features*

In [7]:
items_addtl = item_features[['overview', 'original_language', 'runtime', 'vote_average', 'vote_count', "movie_id"]].copy()

*4. Merge the two dataframes*

In [8]:
items = pd.merge(left=items_main, right=items_addtl, on="movie_id", how="left")

In [9]:
items.head()

,movie_id,title,genre_unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,SciFi,Thriller,War,Western,year,overview,original_language,runtime,vote_average,vote_count
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,1995,"Led by Woody, Andy's toys live happily in his ...",en,81.0,7.9,10878.0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,1,0,0,1995,James Bond must unmask the mysterious head of ...,en,130.0,6.8,2037.0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,1,0,0,1995,It's Ted the Bellhop's first night on the job....,en,98.0,6.1,1251.0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,1995,Chili Palmer is a Miami mobster who gets sent ...,en,105.0,6.5,501.0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,1,0,0,1995,An agoraphobic psychologist and a female detec...,en,124.0,6.5,424.0


In [10]:
items.overview.isna().sum()

18

In [11]:
items.overview.fillna("None", inplace=True)

In [12]:
df_items = pd.read_csv("data/items.csv")

In [13]:
df_items['cleaned_title'] = df_items['title'].apply(remove_year)
df_items.head()

,movie_id,title,genre_unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Thriller,War,Western,year,overview,original_language,runtime,vote_average,vote_count,cleaned_title
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,1995.0,"Led by Woody, Andy's toys live happily in his ...",en,81.0,7.9,10878.0,Toy Story
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,1,0,0,1995.0,James Bond must unmask the mysterious head of ...,en,130.0,6.8,2037.0,GoldenEye
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,1,0,0,1995.0,It's Ted the Bellhop's first night on the job....,en,98.0,6.1,1251.0,Four Rooms
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,1995.0,Chili Palmer is a Miami mobster who gets sent ...,en,105.0,6.5,501.0,Get Shorty
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,1,0,0,1995.0,An agoraphobic psychologist and a female detec...,en,124.0,6.5,424.0,Copycat


### Getting the sentence vector

In [14]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [15]:
doc = nlp(items["overview"][0])

In [16]:
doc.vector

array([-1.6682576 ,  0.08209237, -1.8998522 , -0.57894546,  3.637206  ,
        0.46126536,  0.20418729,  3.307543  ,  1.0883235 , -0.9990239 ,
        3.715574  ,  0.8342156 , -2.6748798 ,  1.082945  ,  1.0651544 ,
        1.2832453 , -0.37863958, -0.07726289,  0.31270948, -1.385957  ,
        2.1643152 , -1.409468  , -0.0342092 , -0.88258064,  0.66496366,
       -0.22921385, -3.0330706 , -0.5107614 ,  0.83954465,  1.8079745 ,
       -0.6356474 ,  0.06916294, -0.1220132 , -1.0252012 , -3.5674474 ,
       -0.32074094, -0.9797936 ,  1.7485722 , -0.36911505,  0.14500082,
        0.4732576 ,  0.21603526,  0.12897743,  0.2927039 , -1.0907979 ,
        1.0357541 , -0.8144599 , -1.6231511 , -0.12688123,  2.5538268 ,
       -1.6596926 ,  1.6522672 , -0.5406606 , -2.2643535 ,  0.3958007 ,
       -0.511619  , -0.49368063, -0.11880509,  0.5386718 , -0.18335347,
       -0.51058096, -1.1527691 ,  0.6106561 , -0.7836873 ,  2.5334463 ,
        1.5647869 , -1.6526694 , -2.130539  ,  0.8158034 ,  0.78

In [17]:
def word_vec(sentence):
    doc = nlp(sentence)
    return doc.vector

In [18]:
%%time
overview_embedding = items["overview"].apply(word_vec)

Wall time: 3min 16s


In [19]:
overview_embedding = overview_embedding.to_list()

In [20]:
overview_embedding_list = []
for vec in overview_embedding:
    overview_embedding_list.append(vec.tolist())

In [21]:
len(overview_embedding_list)

1746

In [22]:
overview_embedding_df = pd.DataFrame(overview_embedding_list)

In [23]:
overview_embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.668258,0.082092,-1.899852,-0.578945,3.637206,0.461265,0.204187,3.307543,1.088323,-0.999024,...,-0.008009,-0.938436,-0.408866,0.054046,-1.415393,0.261185,-0.916689,-1.189812,-1.937821,-0.133747
1,-2.191469,0.074849,-1.251624,1.047345,5.300576,0.019676,0.971809,3.437134,0.696242,-0.539811,...,0.045850,-0.212353,-0.292129,1.653200,-1.479490,0.741954,0.323610,-2.008511,-1.147235,0.422195
2,-0.348014,1.825774,-2.712861,-0.789508,5.125006,0.652385,0.293594,4.687879,0.155474,0.518683,...,0.985423,-1.384183,1.144868,0.452811,-2.827790,1.156295,0.888090,-0.011078,-2.181588,1.911778
3,-2.298307,0.527417,-2.091048,-0.578071,3.678412,-0.044843,0.907063,3.620301,0.787013,-0.807374,...,0.205412,-1.575186,0.487452,-0.045207,-1.349275,0.439918,1.038211,0.148759,-2.477566,1.225671
4,-1.722893,1.267963,-2.099204,0.110817,4.552235,0.197369,2.252256,3.794675,0.768062,-0.360550,...,-0.518746,0.685921,-0.308899,0.812863,-1.295362,1.145068,0.450276,-0.127351,-3.191836,0.640060


In [24]:
items.columns

Index(['movie_id', 'title', 'genre_unknown', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'SciFi', 'Thriller', 'War', 'Western', 'year', 'overview',
       'original_language', 'runtime', 'vote_average', 'vote_count'],
      dtype='object')

In [25]:
item_similarity_df = pd.concat([
    items[['movie_id', 'genre_unknown', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'SciFi', 'Thriller', 'War', 'Western']], 
    overview_embedding_df],
    axis=1)

In [26]:
item_similarity_df.head()

,movie_id,genre_unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,290,291,292,293,294,295,296,297,298,299
0,1,0,0,0,1,1,1,0,0,0,...,-0.008009,-0.938436,-0.408866,0.054046,-1.415393,0.261185,-0.916689,-1.189812,-1.937821,-0.133747
1,2,0,1,1,0,0,0,0,0,0,...,0.045850,-0.212353,-0.292129,1.653200,-1.479490,0.741954,0.323610,-2.008511,-1.147235,0.422195
2,3,0,0,0,0,0,0,0,0,0,...,0.985423,-1.384183,1.144868,0.452811,-2.827790,1.156295,0.888090,-0.011078,-2.181588,1.911778
3,4,0,1,0,0,0,1,0,0,1,...,0.205412,-1.575186,0.487452,-0.045207,-1.349275,0.439918,1.038211,0.148759,-2.477566,1.225671
4,5,0,0,0,0,0,0,1,0,1,...,-0.518746,0.685921,-0.308899,0.812863,-1.295362,1.145068,0.450276,-0.127351,-3.191836,0.640060


# Build nearest neighbor model

In [35]:
from reco.recommend import get_similar
from sklearn.neighbors import NearestNeighbors, VALID_METRICS
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from reco.preprocess import encode_user_item_withencoder

In [36]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [37]:
def get_similar(embedding, k):
    model_similar_items = NearestNeighbors(n_neighbors=k, metric="cosine",
                                           algorithm="brute").fit(embedding)
    distances, indices = model_similar_items.kneighbors(embedding)
    
    return distances, indices

In [38]:
%%time
item_distances, item_similar_indices = get_similar(overview_embedding_df, 5)

Wall time: 437 ms


In [39]:
item_similar_indices

array([[   0,  731, 1089,  220, 1460],
       [   1,  249,  223,  655,  305],
       [   2,  996,  813, 1354, 1676],
       ...,
       [1472, 1743, 1473, 1742,   35],
       [1744,  188,  533, 1181, 1141],
       [1745,  331, 1619, 1646, 1615]], dtype=int64)

In [40]:
df_ratings = pd.read_csv("data/ratings.csv")

In [41]:
DATA, user_encoder, item_encoder = encode_user_item_withencoder(df_ratings, "user_id", "movie_id", "rating", "unix_timestamp")

Number of users:  943
Number of items:  1682


In [42]:
def show_similar(item_name, item_similar_indices,item_encoder, df_items):
    
    # Get the item index from the item name
    item_index = df_items[df_items['cleaned_title'] == item_name]['movie_id'].values[0]
    # Encode the item index using the item encoder
    item_encoded = item_encoder.transform([item_index])[0]
        
    movie_ids = item_similar_indices[item_index]
    #movie_ids = item_encoder.inverse_transform(s)

    images = []
    for movie_id in movie_ids:
        img_path = 'data/posters/' + str(movie_id+1) + '.jpg'
        images.append(mpimg.imread(img_path))

    plt.figure(figsize=(20,10))
    columns = 5
    for i, image in enumerate(images):
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

In [43]:
show_similar("Apollo 13", item_similar_indices, item_encoder, df_items)

FileNotFoundError: [Errno 2] No such file or directory: 'data/posters/1331.jpg'